# Aplicaciones con detección de pescados con modelo entrenado para Kaggle Fisheries

In [44]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../videoanalytics/src")
import videoanalytics
videoanalytics.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.0.2'

In [45]:
from datetime import datetime
import glob
import os

## Configuración y preparación

### Chequeo de disponibilidad de HW / GPU

In [46]:
!nvidia-smi

Sat Sep 18 22:35:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   67C    P0    N/A /  N/A |    637MiB /  4046MiB |     42%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Detectadas por tensorflow en conda (FIXME).

In [4]:
import tensorflow as tf
print("GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles:  0


### Datos de entrada y modelos

In [38]:
WORKSPACE_PATH="../"
WORKSPACE_DATA_PATH=WORKSPACE_PATH+"/data/"
INPUT_VIDEO_PATH=WORKSPACE_DATA_PATH+"media/fishcount/input_videos/"
!ls {INPUT_VIDEO_PATH}

001.mp4  004.mp4  007.mp4  10.mp4  13.mp4  16.mp4  19.mp4
002.mp4  005.mp4  008.mp4  11.mp4  14.mp4  17.mp4
003.mp4  006.mp4  009.mp4  12.mp4  15.mp4  18.mp4


Repositorio de modelos (local).

In [15]:
MODELS_PATH=WORKSPACE_DATA_PATH+"models/"
!ls {MODELS_PATH}

coco		      kaggle_fisheries	     mirtar-yolo  yolov4-416-tf
feature_extractor_ll  kaggle-fisheries-yolo  README.md	  yolov4-416-tf.tar.gz


In [16]:
!tree {MODELS_PATH}/kaggle-fisheries-yolo

..//data/models//kaggle-fisheries-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── kaggle-fisheries-yolo4.cfg
├── kaggle-fisheries-yolo4.weights
├── obj.names
├── README.md
└── yolo4-model-performance-report.txt

4 directories, 8 files


In [17]:
!tree {MODELS_PATH}/mirtar-yolo

..//data/models//mirtar-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── mirtar-yolo4.cfg
├── mirtar-yolo4.weights
├── obj.names
├── README.md
└── yolo4-model-performance-report.txt

4 directories, 8 files


## 1. Generación de detecciones

### 1.1 Pipeline

In [18]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

def make_detection_pipeline(input_video_filename,
                            output_csv_filename,
                            output_video_filename,
                            detector_model_weights_filename,
                            detector_model_classes_filename,
                            start_frame=0,
                            max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )
        
    pipeline.add_component( DetectionsCSVWriter("det_csv_writer",context,filename=output_csv_filename) )
    
    pipeline.add_component( DetectionsAnnotator("annotator",
                                                context,
                                                class_names_filename=detector_model_classes_filename,
                                                show_label=True) )
    
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "detector"),
        ("detector", "det_csv_writer"),
        ("detector", "annotator"),
        ("annotator", "writer")
    ])    
    
    return context,pipeline

### 1.2 Batch job

Directorio de resultados.

In [ ]:
INPUT_VIDEOS = [(os.path.basename(x),None) for x in glob.glob(INPUT_VIDEO_PATH+"/*.mp4")]
INPUT_VIDEOS

In [40]:
from os import path

def object_detection_batch_job(input_videos,obj_det_model, output_path,dry_run=False):
    
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        input_video_filename=v[0]        
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_csv_filename=output_path+"job{:d}.csv".format(i)

        output_csv_filename

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]

        detector_model_weights_filename=MODELS_PATH+"/{}/checkpoints/yolo-416/".format(obj_det_model)
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        

        if not dry_run:
            context,pipeline=make_detection_pipeline(
                input_video_filename=INPUT_VIDEO_PATH+input_video_filename,
                output_csv_filename=output_csv_filename,
                output_video_filename=output_video_filename,
                detector_model_weights_filename=detector_model_weights_filename,
                detector_model_classes_filename=detector_model_classes_filename,
                start_frame=0,
                max_frames=None
            )

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
    
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)

### Job con dataset de kaggle-fisheries

~~~
job 0 14.mp4
job 1 19.mp4
job 3 001.mp4
job 4 18.mp4
job 5 002.mp4
job 6 13.mp4
job 7 15.mp4
job 8 12.mp4
job 9 008.mp4
job 10 007.mp4
job 11 17.mp4
job 12 003.mp4 Probar con MIRTAR
job 13 004.mp4
job 14 009.mp4
job 15 006.mp4 
job 16 005.mp4
job 17 16.mp4
job 18 10.mp4
job 19 11.mp4
~~~ 

In [42]:
input_videos = INPUT_VIDEOS
output_base_path="../data/media/fishcount/"
obj_det_model = "kaggle-fisheries-yolo/"
output_path = output_base_path+"detections/"

In [ ]:
#!rm -rf {output_path} 
#!mkdir {output_path}

In [43]:
object_detection_batch_job(input_videos,obj_det_model, output_path)

Inicio de proceso:  2021-09-18 19:45:06.836164
Job 0. Procesando 14.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 133.43728687000112
Video de salida guardado en ../data/media/fishcount/detections/job0.mp4
Job 1. Procesando 19.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 222.1016069010002
Video de salida guardado en ../data/media/fishcount/detections/job1.mp4
Job 2. Procesando 001.mp4.


...

Tiempo total de ejecución [s]: 137.7099794300011
Video de salida guardado en ../data/media/fishcount/detections/job16.mp4
Job 17. Procesando 10.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 326.9651539280021
Video de salida guardado en ../data/media/fishcount/detections/job17.mp4
Job 18. Procesando 11.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 296.18042221200085
Video de salida guardado en ../data/media/fishcount/detections/job18.mp4
Fin del proceso: 2021-09-18 22:30:45.064736
Tiempo transcurrido: 2:45:38.228572


### Job con dataset Mirta-R

In [52]:
input_videos = [
    ("004.mp4",None)
]
output_base_path="../data/media/fishcount/"
obj_det_model = "mirtar-yolo/"
output_path = output_base_path+"detections-mirtar/"

In [ ]:
# Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [54]:
object_detection_batch_job(input_videos,obj_det_model, output_path)

Inicio de proceso:  2021-09-18 23:04:28.463624
Job 0. Procesando 004.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 909.8754260800015
Video de salida guardado en ../data/media/fishcount/detections-mirtar/job0.mp4
Fin del proceso: 2021-09-18 23:20:12.741677
Tiempo transcurrido: 0:15:44.278053


## 2. Seguimiento

### 2.1 Pipeline

In [181]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter

from videoanalytics.pipeline.sinks.object_detection import ObjectDetectorCSV

from videoanalytics.pipeline.sinks.object_tracking import TrackedObjectsAnnotator, TrackedObjectsCSVWriter
from videoanalytics.pipeline.sinks.object_tracking.sort import SORT
from videoanalytics.pipeline.sinks.object_tracking.deepsort import DeepSORT

from videoanalytics.pipeline.sinks.roi import ROIView, ROIPresenceCounter
from videoanalytics.pipeline.sinks.visualization import TextOverlay

def make_tracking_pipeline( input_video_filename,
                            output_track_csv_filename,
                            output_video_filename,                            
                            detector_precalc_filename,
                            detector_model_classes_filename,
                            roi_definition_filename,
                            tracker_type="SORT",
                            start_frame=0,
                            max_frames=None,tracker_args={}):
    context = {
        "TRACKED_OBJS":[]
    }
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    

    pipeline.add_component( ObjectDetectorCSV("detector_precalc",
                                               context,
                                               filename = detector_precalc_filename) )    
        
    
    def make_sort_tracker(tracker_args):
        print("Making SORT")
        return SORT("tracker", context, **tracker_args)
    
    def make_deep_sort_tracker(tracker_args):
        print("Making DeepSORT")
        return DeepSORT("tracker", context, **tracker_args)
    
    tracker_factory = {
        "SORT": make_sort_tracker,
        "DEEPSORT": make_deep_sort_tracker
    }
    
    pipeline.add_component( tracker_factory[tracker_type](tracker_args) )        
    
    pipeline.add_component( TrackedObjectsAnnotator("annotator",context) )
    pipeline.add_component( TrackedObjectsCSVWriter("tracker_csv_writer",context, 
                                                    filename = output_track_csv_filename) )
    
    # ROIs view + contador
    pipeline.add_component( ROIView("roi_view",context,filename=roi_definition_filename,alpha=0.6 ) )
    #pipeline.add_component( ROIPresenceCounter("roi_counter",context,filename=roi_definition_filename) )
    
    
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "roi_view"),
        ("roi_view", "detector_precalc"),
        ("detector_precalc", "tracker"),
        ("tracker", "tracker_csv_writer"),
        ("tracker", "annotator"),
        ("annotator", "writer")
    ])   
    
    return context,pipeline

### 2.2 Batch

In [129]:
DEEPSORT_MODEL_FILENAME = MODELS_PATH+"/feature_extractor_mars/mars-small128.pb"

In [178]:
from os import path

def object_tracking_batch_job(input_videos,obj_det_model,csv_precalc_path,output_path,roi_path,
                              tracker_type,tracker_args,dry_run=False):
    
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        print(v[1])
        input_video_filename=v[0]        
        csv_precalc_detections_filename=csv_precalc_path+"/"+v[1] 
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_track_csv_filename=output_path+"job{:d}-sort.csv".format(i)        
        roi_filename = roi_path+"/"+v[2] 

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]
        
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        if not dry_run:
            context,pipeline=make_tracking_pipeline(
                input_video_filename=INPUT_VIDEO_PATH+input_video_filename,
                output_track_csv_filename=output_track_csv_filename,
                output_video_filename=output_video_filename,
                detector_precalc_filename=csv_precalc_detections_filename,
                detector_model_classes_filename=detector_model_classes_filename,
                roi_definition_filename = roi_path+"/"+v[2],
                tracker_type=tracker_type,
                start_frame=0,
                max_frames=None,
                tracker_args=tracker_args
            )

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
    
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)
    
    # Devolver último pipeline y contexto
    return pipeline,context

~~~
job 0 14.mp4
job 1 19.mp4
job 2 001.mp4
job 3 18.mp4
job 4 002.mp4 Contar atunes
job 5 13.mp4  Contar atunes (top fija)
job 6 15.mp4
job 7 12.mp4 Probar con MIRTAR
job 8 008.mp4
job 9 007.mp4
job 10 17.mp4
job 11 003.mp4
job 12 004.mp4 
job 13 009.mp4
job 14 006.mp4 
job 15 005.mp4
job 16 16.mp4
job 17 10.mp4
job 18 11.mp4
~~~ 

#### 2.2.1 SORT

In [188]:
input_videos = [
    ("13.mp4","job5.csv","roi13.json"),
    ("001.mp4","job2.csv","roi003.json"),    
    ("002.mp4","job4.csv","roi003.json"),    
    ("003.mp4","job11.csv","roi003.json"),    
]
output_base_path="../data/media/fishcount/"
csv_precalc_path="../data/media/fishcount/detections/"
roi_path="../data/media/fishcount/rois/"
obj_det_model="kaggle-fisheries-yolo"
tracker_type="SORT"
tracker_args= {
    "max_age":10,
    "min_hits": 1,
    "iou_threshold":0.05
}
output_path = output_base_path+"/trackings-sort/".format(
    *list(tracker_args.values()))
object_tracking_batch_job(input_videos,obj_det_model,csv_precalc_path,output_path,roi_path,
                          tracker_type,tracker_args,dry_run=False)

Inicio de proceso:  2021-09-19 01:44:21.232306
job5.csv
Job 0. Procesando 13.mp4.
Making SORT


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 6.1426316579963895
Video de salida guardado en ../data/media/fishcount//trackings-sort/job0.mp4
job2.csv
Job 1. Procesando 001.mp4.
Making SORT


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 7.132599685995956
Video de salida guardado en ../data/media/fishcount//trackings-sort/job1.mp4
job4.csv
Job 2. Procesando 002.mp4.
Making SORT


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 6.763903895000112
Video de salida guardado en ../data/media/fishcount//trackings-sort/job2.mp4
job11.csv
Job 3. Procesando 003.mp4.
Making SORT


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 4.6920959720009705
Video de salida guardado en ../data/media/fishcount//trackings-sort/job3.mp4
Fin del proceso: 2021-09-19 01:44:49.111583
Tiempo transcurrido: 0:00:27.879277


#### 2.2.2 DEEPSORT

In [174]:
tracker_args = {"model_filename":DEEPSORT_MODEL_FILENAME}
input_videos = [
    ("13.mp4","job5.csv")
]
output_base_path="../data/media/fishcount/"
csv_precalc_path="../data/media/fishcount/detections/"
output_path = output_base_path+"/trackings-deepsort/"
obj_det_model="kaggle-fisheries-yolo"
tracker_type="DEEPSORT"
tracker_args= {
    "model_filename": DEEPSORT_MODEL_FILENAME,
    "max_cosine_distance": 0.4,
    "nn_budget": None,
    "max_iou_distance": 0.7,
    "max_age": 60,
    "n_init": 3
}
object_tracking_batch_job(input_videos,obj_det_model,csv_precalc_path,output_path,
                          tracker_type,tracker_args,dry_run=False)

Inicio de proceso:  2021-09-19 00:47:26.139173
job5.csv
Job 0. Procesando 13.mp4.
Making DeepSORT


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 6.216584733003401
Video de salida guardado en ../data/media/fishcount//trackings-deepsort/job0.mp4
Fin del proceso: 2021-09-19 00:47:32.819493
Tiempo transcurrido: 0:00:06.680320
